In [1]:
import numpy as np

from qibo.symbols import Z
from qibo import hamiltonians

from qiboml.models.encoding_decoding import *
from qiboml.models.ansatze import *

def run_model(model, x):
    for layer in model:
        x = layer.forward(x)
    return x

In [2]:
# binary data
model = [BinaryEncodingLayer(nqubits=3), ReuploadingLayer(nqubits=3, qubits=(0,2)), QuantumDecodingLayer(nqubits=3, qubits=(1,0))]
data = np.array([[0,0,0], [0,1,0], [1,1,0]])

for x in data:
    print(run_model(model, x).probabilities())


tf.Tensor([1. 0. 0. 0. 0. 0. 0. 0.], shape=(8,), dtype=float64)
tf.Tensor([0. 0. 1. 0. 0. 0. 0. 0.], shape=(8,), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 1. 0.], shape=(8,), dtype=float64)


In [3]:
# real data
nqubits = 4
observable = hamiltonians.SymbolicHamiltonian(np.prod([Z(q) for q in range(nqubits)]))
model = [PhaseEncodingLayer(nqubits, qubits=range(3)), ReuploadingLayer(nqubits), ExpectationLayer(nqubits, observable=observable)]
data = np.pi * np.random.randn(3**2).reshape(3, 3)

for x in data:
    print(run_model(model, x))

[Qibo 0.2.9|INFO|2024-06-13 12:27:00]: Using qibojit (numba) backend on /CPU:0


1.0
1.0
1.0


In [10]:
# pytorch interface
import torch
import qiboml.models.pytorch as pt

model = torch.nn.Sequential(
    torch.nn.Linear(128,5),
    torch.nn.Sigmoid(),
    pt.BinaryEncodingLayer(5),
    pt.ReuploadingLayer(nqubits=5, qubits=(0,2,4)),
    pt.QuantumDecodingLayer(nqubits=5, qubits=reversed(range(5)))
)

print(f"> Model: {model}")
print("> Parameters")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.shape)

data = torch.randn(3,128)
print("> Outputs")
for x in data:
    print(model(x).probabilities())


> Model: Sequential(
  (0): Linear(in_features=128, out_features=5, bias=True)
  (1): Sigmoid()
  (2): BinaryEncodingLayer(nqubits=5, qubits=[0, 1, 2, 3, 4], circuit=<qibo.models.circuit.Circuit object at 0x7f8ae19df5e0>, initial_state=None, backend=tensorflow)
  (3): ReuploadingLayer(nqubits=5, qubits=(0, 2, 4), circuit=<qibo.models.circuit.Circuit object at 0x7f8b2ebe1630>, initial_state=None, backend=tensorflow)
  (4): QuantumDecodingLayer(nqubits=5, qubits=<range_iterator object at 0x7f8ae189a400>, circuit=<qibo.models.circuit.Circuit object at 0x7f8ae18985b0>, initial_state=None, backend=tensorflow, nshots=1000)
)
> Parameters
0.weight torch.Size([5, 128])
0.bias torch.Size([5])
3.ReuploadingLayer torch.Size([6, 1])
> Outputs
tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.], shape=(32,), dtype=float64)
tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.], shape=(32

In [21]:
# keras/tensorflow interface
import keras
import tensorflow as tf
import qiboml.models.keras as ks

model = keras.Sequential([
    keras.layers.Dense(5, activation="sigmoid", name="dense"),
    ks.BinaryEncodingLayer(5, name="encoding"),
    ks.ReuploadingLayer(nqubits=5, qubits=(0,2,4), name="reuploading"),
    ks.QuantumDecodingLayer(nqubits=5, qubits=reversed(range(5)), name="decoding")
])

print(f"> Model: {model}")

data = tf.random.uniform((3,128))
print("> Outputs")
for x in data:
    x = tf.expand_dims(x, axis=0)
    print(model(x).probabilities())



> How do you register parameters with Keras?
> Model: <Sequential name=sequential_9, built=False>
> Outputs


RuntimeError: Exception encountered when calling BinaryEncodingLayer.call().

[1mCould not automatically infer the output shape / dtype of 'encoding' (of type BinaryEncodingLayer). Either the `BinaryEncodingLayer.call()` method is incorrect, or you need to implement the `BinaryEncodingLayer.compute_output_spec() / compute_output_shape()` method. Error encountered:

Tried to convert 'input' to a tensor and failed. Error: None values not supported.[0m

Arguments received by BinaryEncodingLayer.call():
  • args=('<KerasTensor shape=(1, 5), dtype=float32, sparse=False, name=keras_tensor_14>',)
  • kwargs=<class 'inspect._empty'>